In [1]:
import csv
import psycopg2
from psycopg2 import sql
from datetime import datetime
import logging
import os
from dotenv import load_dotenv

In [2]:
from google.cloud import secretmanager
def get_secret(secret_id, project_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("utf-8")

In [3]:
PROJECT_ID = "cloud-computing-project-458205"
load_dotenv()
DB_NAME = os.getenv("DB_NAME", "database-instance")
DB_USER = get_secret("DB_USER", PROJECT_ID).replace("\n", "")
DB_PASSWORD = get_secret("DB_PASSWORD", PROJECT_ID).replace("\n", "")
DB_HOST = get_secret("DB_HOST", PROJECT_ID).replace("\n", "")
DB_PORT = os.getenv("DB_PORT", "5432")

In [4]:
from psycopg2.extras import DictCursor
conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            cursor_factory=DictCursor,  # Use DictCursor to access columns by name
        )
conn

<connection object at 0x7a0029f2efc0; dsn: 'user=app-engine-user password=xxx dbname=database-instance host=34.140.62.43 port=5432', closed: 0>

In [27]:
create_table_query = """
    DROP TABLE IF EXISTS fingerprints;
    CREATE TABLE fingerprints (
        fingerprint_id SERIAL PRIMARY KEY,
        song_id INTEGER NOT NULL REFERENCES tracks(track_id) ON DELETE CASCADE,
        hash TEXT NOT NULL,
        timestamp FLOAT NOT NULL,
        created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
        UNIQUE (song_id, hash, timestamp)
    );
    CREATE INDEX idx_fingerprints_hash ON fingerprints(hash);
    """

In [28]:
with conn.cursor() as cursor:
    cursor.execute(create_table_query)
    conn.commit()
    print("Fingerprints table created successfully.")

Fingerprints table created successfully.


In [5]:
from src.abracadabra.GCPFingerprintDB import GCPFingerprintDB

db = GCPFingerprintDB()
db.show_table()

(1, 'In Da Club')
(2, 'In Da Club - Single Version / Edit')
(3, 'Just A Lil Bit')
(4, 'P.I.M.P.')
(5, 'Window Shopper')
(6, 'Back In Black')
(7, 'Dirty Deeds Done Dirt Cheap')
(8, 'Heatseeker')
(9, 'High Voltage')
(10, 'Highway to Hell')
(11, "It's a Long Way to the Top (If You Wanna Rock 'N' Roll)")
(12, 'Jailbreak')
(13, "Rock and Roll Ain't Noise Pollution")
(14, 'Shoot to Thrill')
(15, 'T.N.T.')
(16, 'Thunderstruck')
(17, 'Who Made Who')
(18, 'You Shook Me All Night Long')
(19, 'Chasing Pavements')
(20, "Don't You Remember")
(21, 'Easy On Me')
(22, "He Won't Go")
(23, 'Hello')
(24, 'Love In The Dark')
(25, 'Make You Feel My Love')
(26, 'Million Years Ago')
(27, 'One and Only')
(28, 'Remedy')
(29, 'Rolling in the Deep')
(30, 'Rumour Has It')
(31, 'Send My Love (To Your New Lover)')
(32, 'Set Fire to the Rain')
(33, 'Skyfall')
(34, 'Someone Like You')
(35, 'Sweetest Devotion')
(36, 'Take It All')
(37, 'Turning Tables')
(38, 'Water Under The Bridge')
(39, 'When We Were Young')
(40, 'A

In [6]:
from src.abracadabra.recognize import index_all_songs
index_all_songs(db_type="gcp")

  Indexed Fallin' (45/1026)
  Indexed Girl On Fire (46/1026)
Indexed Hallelujah (47/1026)                            
  Indexed Holy War (48/1026)
  Indexed If I Ain't Got You (49/1026)
  Indexed More Than We Know (50/1026)
  Indexed No One (51/1026)


  Indexed One Thing (52/1026)
  Indexed Superwoman (53/1026)
  Indexed Tears Always Win (54/1026)
  Indexed Work On It (55/1026)
  Indexed No More "I Love You's" (56/1026)
  Indexed Arabella (57/1026)
  Indexed Do I Wanna Know? (58/1026)
  Indexed I Bet You Look Good On The Dancefloor (59/1026)
  Indexed Why'd You Only Call Me When You're High? (60/1026)
  Indexed Addicted To You (61/1026)
  Indexed Broken Arrows (62/1026)
  Indexed Fade Into Darkness - Vocal Radio Mix (63/1026)
  Indexed For A Better Day (64/1026)
  Indexed Heaven (65/1026)
  Indexed Hey Brother (66/1026)
  Indexed Levels - Radio Edit (67/1026)
  Indexed Silhouettes - Original Radio Edit (68/1026)
  Indexed The Nights (69/1026)
  Indexed Waiting For Love (70/1026)
  Indexed Wake Me Up (71/1026)
  Indexed Wake Me Up - Radio Edit (72/1026)
  Indexed You Make Me (73/1026)
  Indexed Butter (74/1026)
  Indexed Dynamite (75/1026)
  Indexed BREAK MY SOUL (76/1026)
  Indexed Best Thing I Never Had (77/1026)
 Indexed Broken-He

KeyboardInterrupt: 

In [1]:
from src.abracadabra.recognize import index_all_songs
index_all_songs(db_type="gcp")

In [11]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM tracks where artist_names LIKE 'PSY';")
    rows = cur.fetchall()
    for row in rows:
        print(row)

In [8]:
from src.abracadabra.recognize import recognize_song
match = recognize_song("../songs/Adele - Someone Like You (Official Music Video).m4a", db)

if match:
    print(f"Matched: {match[0]} (score: {match[1]})")
    # track_name, artist_names, album_name, youtube_url = db.check_song_info(match[0])
    track_name, youtube_url = db.check_song_info(match[0])
    print(f"Track Name: {track_name}")
    print(f"Artist Names: {artist_names}")
    print(f"Album Name: {album_name}")
    print(f"YouTube URL: {youtube_url}")
else:
    print("No match found.")

Matched: 34 (score: 25)
Track Name: Someone Like You


NameError: name 'artist_names' is not defined